## Initiation

In [ ]:
import pytest
from src.tasks import app, worth

@pytest.fixture
def test_app():
    app.config.update({"TESTING": True})
    return app

@pytest.fixture
def client(test_app):
    return test_app.test_client()

body = {
    "title": "test_task",
    "description": "this is a test task",
    "priority": 2,
    "difficulty": 2,
    "due_date": "2025-10-20"
}

## Étape 1 : Tests Fonctionnels et Boîte Noire

In [ ]:
def test_add_task(client):
    with app.app_context():
        response = client.post('/tasks', json=body)
        assert response.status_code == 201
        task_data = response.get_json()
        assert task_data["title"] == body["title"]
        assert task_data["priority"] == body["priority"]
        assert task_data["difficulty"] == body["difficulty"]

def test_complete_task(client):
    with app.app_context():
        response = client.post('/tasks', json=body)
        task_id = response.get_json()["id"]

        response = client.post(f"/tasks/{task_id}/complete")
        assert response.status_code == 200
        assert response.get_json()["message"] == "Tâche marquée comme terminée"

def test_cleanup_tasks(client):
    with app.app_context():
        past_task = body.copy()
        past_task["due_date"] = "2022-01-01"
        client.post("/tasks", json=past_task)

        response = client.delete("/tasks/cleanup")
        assert response.status_code == 200
        assert "tâches obsolètes ou complétées supprimées" in response.get_json()["message"]

## Étape 2 : Tests Boîte Blanche et Couverture de Code

In [ ]:
def test_is_date_and_in_past():
    assert is_date_and_in_past("2020-01-01") == True
    assert is_date_and_in_past("2099-01-01") == False
    assert is_date_and_in_past(None) == False

def test_worth_calculation():
    assert worth(priority=2, difficulty=3, due_date="2020-01-01") == 30  # Past date, score halved
    assert worth(priority=2, difficulty=3, due_date="2099-01-01") == 60  # Future date, full score

## Étape 3 : Utilisation de Mocks

In [ ]:
def test_task_lifecycle(client):
    with app.app_context():
        response = client.post("/tasks", json=body)
        task_id = response.get_json()["id"]

        response = client.get("/tasks/active")
        active_tasks = response.get_json()
        assert any(task["id"] == task_id for task in active_tasks), "Task should be active after creation."

        response = client.post(f"/tasks/{task_id}/complete")
        assert response.status_code == 200
        assert response.get_json()["message"] == "Tâche marquée comme terminée"

        response = client.get("/tasks/active")
        active_tasks = response.get_json()
        assert not any(task["id"] == task_id for task in active_tasks), "Completed task should not appear in active tasks."

        response = client.delete("/tasks/cleanup")
        assert response.status_code == 200

        response = client.get(f"/tasks/{task_id}")
        assert response.status_code == 404, "Task should not exist after cleanup."

mock permet d'isoler des parties du code, ce qui autorise de tester des fonction non implementer et de les tester dans d'autres circonstances

## Étape 4 : Tests Bout-en-Bout (E2E)

In [ ]:
def test_total_score(client):
    with app.app_context():
        response = client.post("/tasks", json=body)
        task_id1 = response.get_json()["id"]
        client.post(f"/tasks/{task_id1}/complete")

        another_task = body.copy()
        another_task["title"] = "another_test_task"
        response = client.post("/tasks", json=another_task)
        task_id2 = response.get_json()["id"]
        client.post(f"/tasks/{task_id2}/complete")

        response = client.get("/scores/total")
        assert response.status_code == 200
        assert response.get_json()["total_score"] > 0 

## Étape 5 : Tests d'Intégration et Validation de l'État de la Base de Données

In [ ]:
def test_total_score(client):
    with app.app_context():
        response = client.post("/tasks", json=body)
        task_id1 = response.get_json()["id"]
        client.post(f"/tasks/{task_id1}/complete")

        another_task = body.copy()
        another_task["title"] = "another_test_task"
        response = client.post("/tasks", json=another_task)
        task_id2 = response.get_json()["id"]
        client.post(f"/tasks/{task_id2}/complete")

        response = client.get("/scores/total")
        assert response.status_code == 200
        assert response.get_json()["total_score"] > 0

## Étape 6 : Génération de Rapport de Tests

pour voir la couverture des test on genere un rapport html avec `pytest --cov=src --cov-report=html`,

- il n'est pas possible de tester le cas de "pas de tache active", car il n'y a pas de moyen de metre la base de donnée a zero
- ce n'est pas possible de tester init_db() ou app.run()

[lien git](https://github.com/Corentin-Rougeon/methodologie-du-test/tree/main)